In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price active'.split()

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 5, 14), datetime.date(2022, 5, 13))

In [2]:
#yesterday = date(2021, 5, 14)
yesterday

datetime.date(2022, 5, 13)

### Matching check with profits in portmy database 22-05-14 with 2022-q1 data

In [3]:
format_dict = {
    'qty':'{:,}',
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%',  
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',    
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
}

In [4]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.dtypes

id                 int64
name              object
max_price        float64
min_price        float64
status            object
buy_target       float64
sell_target      float64
volume           float64
beta             float64
cost             float64
qty                int64
buy_spread         int64
sell_spread        int64
available_qty      int64
bl                 int64
sh                 int64
reason            object
market            object
dtype: object

In [5]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(34, 23)

In [6]:
df_merge = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge.shape

(57, 41)

In [7]:
names = df_merge[df_merge._merge == 'right_only']['name']
names

Series([], Name: name, dtype: object)

In [8]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

''

In [9]:
sql = """
SELECT id+9000 AS id, name, market, max_price, min_price, daily_volume AS volume, beta 
FROM stocks
WHERE name IN (%s)"""
sql = sql % in_p
my_stocks = pd.read_sql(sql, conmy)
my_stocks

,id,name,market,max_price,min_price,volume,beta


In [103]:
my_stocks.dtypes

id           object
name         object
market       object
max_price    object
min_price    object
volume       object
beta         object
dtype: object

In [104]:
my_stocks['status'] = 'O'
my_stocks['buy_target'] = 0.00
my_stocks['sell_target'] = 0.00
my_stocks['cost'] = 0.00
my_stocks['qty'] = 0
my_stocks['buy_spread'] = -4
my_stocks['sell_spread'] = 4
my_stocks['available_qty'] = 0
my_stocks['bl'] = 0
my_stocks['sh'] = 0
my_stocks['reason'] = 'XXX'
my_stocks.dtypes

id                object
name              object
market            object
max_price         object
min_price         object
volume            object
beta              object
status            object
buy_target       float64
sell_target      float64
cost             float64
qty                int64
buy_spread         int64
sell_spread        int64
available_qty      int64
bl                 int64
sh                 int64
reason            object
dtype: object

In [105]:
rearrange_col = 'id name max_price min_price status buy_target sell_target volume beta cost qty buy_spread sell_spread available_qty bl sh reason market'.split()
rearrange_col

['id',
 'name',
 'max_price',
 'min_price',
 'status',
 'buy_target',
 'sell_target',
 'volume',
 'beta',
 'cost',
 'qty',
 'buy_spread',
 'sell_spread',
 'available_qty',
 'bl',
 'sh',
 'reason',
 'market']

In [106]:
my_stocks = my_stocks[rearrange_col]

In [107]:
rcds = my_stocks.values.tolist()
len(rcds)

0

In [108]:
for rcd in rcds:
    print(rcd)

In [109]:
for rcd in rcds:
    conlite.execute("""INSERT INTO stocks (id, name, max_price, min_price, status, buy_target, sell_target, volume, beta, cost, qty, buy_spread, sell_spread, available_qty, bl, sh, \
reason, market) \
VALUES( ?, ?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?)""", rcd)

In [110]:
sqlDel = """
DELETE FROM stocks
WHERE name IN (%s)"""
sqlDel = sqlDel % in_p
#rp = conlite.execute(sqlDel)
rp.rowcount

11

### End of Process insert profit records to stocks

### Process for stock records that are not in profits table

In [111]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(57, 18)

In [112]:
df_merge2 = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge2.shape

(57, 41)

In [113]:
names = df_merge2[df_merge2._merge == 'left_only']['name']
names

2        ASP
5        DCC
6        DIF
8      HREIT
9      JASIF
10     MAKRO
11       MCS
13       NER
16      SCCC
17      SENA
21       TMT
23     WHART
24       BCH
27    DOHOME
31       ORI
36       STA
37     SYNEX
45       KCE
48       PTT
49     PTTGC
50     RATCH
52       SCC
56        IP
Name: name, dtype: object

In [114]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'DCC', 'DIF', 'HREIT', 'JASIF', 'MAKRO', 'MCS', 'NER', 'SCCC', 'SENA', 'TMT', 'WHART', 'BCH', 'DOHOME', 'ORI', 'STA', 'SYNEX', 'KCE', 'PTT', 'PTTGC', 'RATCH', 'SCC', 'IP'"

In [115]:
sqlDel = """
DELETE FROM stocks
WHERE name IN (%s)
AND status = 'X'
AND available_qty = 0"""
sqlDel = sqlDel % in_p
rp = conlite.execute(sqlDel)
rp.rowcount

0

### End of Process to delete stock record that are not in profits and avalable_qty = 0

In [5]:
sql = '''
SELECT *
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.dtypes

name                object
q4                 float64
q3                 float64
q2                 float64
q1                 float64
dividend           float64
shares               int64
xdate       datetime64[ns]
paiddate    datetime64[ns]
kind                object
actual               int64
dtype: object

In [6]:
dividend.sort_values(['xdate'],ascending=[True]).style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
4,HTC,1.1200,0.0000,0.8300,0.0000,1.9500,0,2021-09-23,2021-09-23,,0
65,LPF,0.1719,0.1894,0.1793,0.1878,0.7284,0,2022-01-28,2022-02-15,,0
31,GLOBAL,0.0000,0.0000,0.0000,0.0000,0.0000,0,2022-02-02,2022-02-02,,0
68,IP,0.0000,0.0000,0.0000,0.0000,0.0000,0,2022-02-02,2022-02-02,,0
74,DOHOME,0.0000,0.0000,0.0000,0.0000,0.0000,"7,200",2022-02-02,2022-02-02,,0
77,STARK,0.0000,0.0000,0.0000,0.0000,0.0000,0,2022-02-02,2022-02-02,,0
78,PTTEP,3.0000,0.0000,2.0000,0.0000,5.0000,0,2022-02-10,2022-04-18,,0
75,SCB,2.6300,0.0000,1.4300,0.0000,4.0600,0,2022-02-14,2022-04-22,,0
40,ADVANC,4.2400,0.0000,3.4500,0.0000,7.6900,0,2022-02-18,2022-04-19,,0
43,GVREIT,0.1950,0.1765,0.1765,0.1915,0.7395,0,2022-02-23,2022-09-03,,0


In [7]:
df_merge = pd.merge(stocks, dividend, on='name', how='outer', indicator=True)
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]
#no_dividend.sort_values(['xdate'],ascending=[True])

,name,buy_target,market,status,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual,_merge


### End of Process

### Consensus table

In [10]:
sql = '''
SELECT * FROM consensus ORDER BY name'''
css = pd.read_sql(sql, conlite)
css

,id,name,price,buy,hold,sell,eps_a,eps_b,pe,pbv,yield,consensus_price,status,stock_id
0,2845,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
1,2846,Directory of C:\A8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
2,2844,Volume Serial Number is C496-DC55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
3,2415,AAV,2.94,0.0,0.0,3.0,-0.36,-0.06,-8.2,2.1,0.0,2.33,Z,NaN
4,2722,ABM,1.37,1.0,0.0,0.0,0.13,0.16,10.5,1.2,3.6,2.51,Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,2753,WP,5.10,1.0,0.0,0.0,0.22,0.36,23.2,2.1,4.9,5.90,Z,NaN
429,2674,WPH,3.26,0.0,0.0,0.0,0.07,0.11,46.6,1.8,0.9,0.00,Z,NaN
430,2648,XO,23.00,3.0,0.0,0.0,1.06,1.16,21.7,7.9,2.8,23.60,Z,NaN
431,2750,ZEN,12.00,3.0,0.0,0.0,-0.29,0.50,-40.9,3.3,0.0,14.57,Z,NaN


In [11]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
df_prf = pd.read_sql(sql, conmy)
df_prf.shape

(34, 23)

In [14]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
df_stk = pd.read_sql(sql, conlite)
df_stk.shape

(57, 18)

In [15]:
df_merge = pd.merge(df_prf,df_stk,on='name',how='outer',indicator=True)
df_merge.shape

(57, 41)

In [16]:
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market,_merge


In [17]:
right_mask= df_merge._merge == 'right_only'
df_merge[right_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market,_merge
34,NaN,ASP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.80,30000,-9,12,30000,0,0,DOS,SET,right_only
35,NaN,DCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.96,60000,-4,4,60000,0,0,CP1S,SET,right_only
36,NaN,DIF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.70,10000,-5,5,50000,0,0,CP1S,SET,right_only
37,NaN,HREIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.95,40100,-5,5,40100,0,12,DOS,SET,right_only
38,NaN,JASIF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.00,10000,-5,4,100000,0,0,ROUND,SET,right_only
39,NaN,MAKRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.40,1500,-8,8,7500,0,0,C40.40,SET,right_only
40,NaN,MCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.40,10000,-5,5,75000,6,3,ROUND,SET,right_only
41,NaN,NER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.45,9000,-13,14,27000,0,0,5pct,SET,right_only
42,NaN,SCCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,171.00,300,-10,9,1200,0,0,RD10pct,SET,right_only
43,NaN,SENA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.48,15000,-8,10,105000,0,0,CP1S,SET,right_only


In [18]:
both_mask= df_merge._merge == 'both'
df_merge[both_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market,_merge
0,2417.0,AIT,2022.0,1.0,1.0,571977.0,455882.0,116095.0,25.47,571977.0,...,0.00,0,-4,4,0,0,0,XXX,SET,both
1,2367.0,ASK,2022.0,1.0,1.0,1277644.0,958779.0,318865.0,33.26,1277644.0,...,0.00,0,-4,4,0,0,0,XXX,SET,both
2,2349.0,BAY,2022.0,1.0,1.0,34707618.0,22512053.0,12195565.0,54.17,34707618.0,...,0.00,0,-4,4,0,0,0,XXX,SET,both
3,2353.0,BBL,2022.0,1.0,1.0,26702007.0,16433170.0,10268837.0,62.49,26702007.0,...,0.00,2400,-9,11,0,0,0,1L,SET50,both
4,2378.0,BCPG,2022.0,1.0,1.0,2850502.0,1861919.0,988583.0,53.09,2850502.0,...,0.00,0,-4,4,0,0,0,XXX,SET100,both
5,2401.0,BDMS,2022.0,1.0,1.0,10040447.0,5984852.0,4055595.0,67.76,10040447.0,...,0.00,4200,-6,7,0,0,0,1L,SET50,both
6,2360.0,BH,2022.0,1.0,1.0,1849587.0,530070.0,1319517.0,248.93,1849587.0,...,0.00,0,-4,4,0,0,0,XXX,SET50,both
7,2371.0,BPP,2022.0,1.0,1.0,5011201.0,3761295.0,1249906.0,33.23,5011201.0,...,0.00,0,-4,4,0,0,0,XXX,SET100,both
8,2357.0,COTTO,2022.0,1.0,1.0,608126.0,482435.0,125691.0,26.05,608126.0,...,0.00,0,-4,4,0,0,0,XXX,SET,both
9,2412.0,ESSO,2022.0,1.0,1.0,7554525.0,1184192.0,6370333.0,537.95,7554525.0,...,0.00,0,-4,4,0,0,0,XXX,SET100,both


In [ ]:
sql = '''
SELECT * FROM SALES ORDER BY name'''
sales = pd.read_sql(sql, conlite)
sales.shape[0]

In [ ]:
sql = '''
DELETE FROM sales
WHERE name = "TCAP"'''
rp = conlite.execute(sql)
rp.rowcount